In [1]:
import pandas as pd
import glob
import sys
from time import sleep
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import numpy
import scipy
import sklearn.datasets
import csv

from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree

In [2]:
TRAIN_PATH = 'train/train'
TEST_PATH = 'test/test'

def load_train(class_name):
    label=0
    if class_name is "pos":
        label=1
    data=[]
    for file in glob.glob(TRAIN_PATH+"/"+class_name+"/*.txt"):
        f = open(file, "r", encoding='utf-8')
        data.append([f.read(),label])
    return data
def load_test():
    data=[]
    for i in range(0,25000):
        file=TEST_PATH+"/"+str(i)+".txt"
        f = open(file, "r", encoding='utf-8')
        data.append([f.read(),i])
    return data

In [3]:
train_data=load_train("pos")+load_train("neg")
print(len(train_data))

25000


In [4]:
test_data=load_test()
print(test_data[0])

['Thinking that it could only get better was the worst assumption I ever made....<br /><br />Drivvle does not describe this movie appropriately enough!<br /><br />Not only is the plot thin, but I get more emotional acting from my pet fish!<br /><br />It was a shame to see Pete Postlethwaite, whom I respect as an actor trying to do the best with the little he had to work with...<br /><br />I think that a cardboard cut out of Stephen Baldwin would have done a better job , and in fact have been more animate.<br /><br />Avoid at all costs! This could really be hazardous to your health!', 0]


In [19]:
def cleanUp(text, custom_stopwords=[]):
    # Initilaise Lemmatizer object:
    lemm = WordNetLemmatizer()
    
    # Load NLTK stopwords:
    my_stopwords = stopwords.words('english') + custom_stopwords
    
    clean_text = ''
    
    words = word_tokenize(text) # word_tokenize() takes care of stripping too.
    
    for word in words:
        w = lemm.lemmatize(word.lower())
        if w not in my_stopwords and len(w)>2:
            clean_text += w + " "
    
    return clean_text

In [20]:
def cleanUp_data(dataset):
    for i in range(0,len(dataset)):
        comment=cleanUp(dataset[i][0])
        dataset[i][0]=comment

In [21]:
cleanUp_data(train_data)

In [22]:
train_set,valid_set=sklearn.model_selection.train_test_split(train_data,train_size=0.8,test_size=0.2,shuffle=True)
train_x=[comment[0] for comment in train_set]
train_y=[comment[1] for comment in train_set]
valid_x=[comment[0] for comment in valid_set]
valid_y=[comment[1] for comment in valid_set]

In [23]:
def LogRegression(x_train, y_train, x_valid, y_valid):
    lg = LogisticRegression().fit(x_train, y_train)
    x_v=vectorizer.transform(x_valid)
    y_pred = lg.predict(x_v)
    print ("LR Accuracy: ", sklearn.metrics.accuracy_score(y_valid, y_pred))
    print (sklearn.metrics.confusion_matrix(y_valid, y_pred, labels=[1, 0]))

In [24]:
def SVM(X,y_train,x_test,y_test):
    clf=SVC(kernel='linear')
    clf.fit(X,y_train)
    X_test=vectorizer.transform(x_test)
    y_pred=clf.predict(X_test)
    print ("SVM Accuracy: ", sklearn.metrics.accuracy_score(y_test, y_pred))
    print (sklearn.metrics.confusion_matrix(y_test, y_pred, labels=[1, 0]))

In [25]:
def DecTree(x_train, y_train, x_test, y_test):
    dt = tree.DecisionTreeClassifier(max_depth=10,min_samples_leaf=2,max_leaf_nodes=300,min_samples_split=2)
    dt.fit(x_train, y_train)
    x_v=vectorizer.transform(x_test)
    y_pred = dt.predict(x_v)
    print ("Decision Tree Accuracy: ", sklearn.metrics.accuracy_score(y_test, y_pred))
    print (sklearn.metrics.confusion_matrix(y_test, y_pred, labels=[1, 0]))

In [26]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(train_x)

In [27]:
tfidf_transformer = TfidfTransformer(use_idf=False)
X=tfidf_transformer.fit_transform(X)

In [28]:
LogRegression(X,train_y,valid_x,valid_y)
print("\n")

LR Accuracy:  0.8654
[[2183  271]
 [ 402 2144]]




In [29]:
SVM(X,train_y,valid_x,valid_y)
print("\n")

SVM Accuracy:  0.8826
[[2229  225]
 [ 362 2184]]




In [78]:
DecTree(X,train_y,valid_x,valid_y)
print("\n")

Decision Tree Accuracy:  0.721
[[2079  461]
 [ 934 1526]]




In [46]:
# new feature 1 -- Bing Liu's Opinion Lexicon
POS_PATH='opinion-lexicon-English/positive-words.txt'
NEG_PATH='opinion-lexicon-English/negative-words.txt'
pos_words = []
neg_words = []
def load_pos(pos_set):
    with open(POS_PATH) as f:
        for line in f:
            pos_set.append(line.rstrip())
    return pos_set

def load_neg(neg_set):
    with open(NEG_PATH) as f:
        for line in f:
            neg_set.append(line.rstrip())
    return neg_set

In [47]:
pos_words = load_pos(pos_words)
neg_words = load_neg(neg_words)
print(pos_words)

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [70]:
def count_pos(c_words):
#     count = 0
#     for word in dataset:
#         if word in pos_words:
#             count+=1
    return len(list(set(c_words) & set(pos_words)))

def count_neg(c_words):
#     count = 0
#     for word in dataset:
#         if word in neg_words:
#             count+=1
#     return count
    return len(list(set(c_words) & set(neg_words)))

In [71]:
# print(train_x[0])
f1_train_x=[[count_pos(comment),count_neg(comment)] for comment in train_x]
# f1_train_y=train_y
# f1_valid_x=[[count_pos(comment[0]),count_neg(comment[0])] for comment in valid_set]
# f1_valid_y=valid_y

In [72]:
f1_train_y=train_y
f1_valid_x=[[count_pos(comment),count_neg(comment)] for comment in valid_x]
f1_valid_y=valid_y

In [73]:
LogRegression(f1_train_x,f1_train_y,f1_valid_x,f1_valid_y)
print("\n")

AttributeError: 'list' object has no attribute 'lower'